<a href="https://colab.research.google.com/github/Priyanshu-Aggarwal8/Pytorch_Learning_Notebooks/blob/main/Pytorch_Modular_Script_TinyVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In this notebook, we will recreate the model we created in the previous notebook, but converting the model into executable python scripts along the way.
# There are some Magic commands, like %%writefile, that are used to convert code to a python file.
# By the end of this notebook, we will have a directory structure, using which we can quickly replicate our model and its functionalities inside another notebook as well as from the CLIs also.

In [ ]:
import os
os.makedirs("going_modular", exist_ok = True)

In [ ]:
import os
import zipfile
from pathlib import Path
import requests

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory already exists")
else:
    print(f"Creating {image_path} directory")
    image_path.mkdir(parents = True, exist_ok = True)

with open(data_path / "pizza_steak_sushi.zip" , "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data")
    f.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping data...")
  zip_ref.extractall(image_path)

os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi directory already exists
Unzipping data...


In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

In [ ]:
from torchvision import datasets, transforms

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [ ]:
class_names = train_data.classes
class_dict = train_data.class_to_idx

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1,
                              num_workers=1,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False)

In [ ]:
img, label = next(iter(train_dataloader))

print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


# Creating `data_setup.py`

The above functionality can all be turned into a script and used everytime we want to load the data. We will now create a script named `data_setup.py` and within this script, create a function `create_dataloaders()`.

In [ ]:
%%writefile going_modular/data_setup.py

import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir : str,
                       test_dir  :str,
                       transform : transforms.Compose,
                       batch_size : int,
                       num_workers = NUM_WORKERS):
    train_data = datasets.ImageFolder(train_dir, transform = transform)
    test_data = datasets.ImageFolder(test_dir, transform = transform)

    class_names = train_data.classes

    train_dataloader = DataLoader(train_data,
                                  shuffle = True,
                                  batch_size = batch_size,
                                  num_workers = num_workers,
                                  pin_memory = True)
    test_dataloader = DataLoader(test_data,
                                 shuffle = False,
                                 batch_size = batch_size,
                                 num_workers = num_workers,
                                 pin_memory = True)

    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


#  Creating `model_builder.py`

In [ ]:
%%writefile going_modular/model_builder.py

import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self, input_shape : int, hidden_units : int, output_shape : int) -> None:
    super().__init__()
    self.conv_blvok_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape, out_channels = hidden_units, kernel_size = 3, stride = 1, padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units, out_channels = hidden_units, kernel_size = 3, stride = 1, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.conv_blvok_2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_units, out_channels = hidden_units, kernel_size = 3, padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units, out_channels = hidden_units, kernel_size = 3, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units * 13 * 13, out_features = output_shape)
    )

  def forward(self, x : torch.Tensor):
    x = self.conv_blvok_1(x)
    x = self.conv_blvok_2(x)
    x = self.classifier(x)
    return x

Overwriting going_modular/model_builder.py


In [ ]:
import torch
from going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(42)

model_1 = model_builder.TinyVGG(input_shape = 3, hidden_units = 10, output_shape = 3).to(device)
model_1

TinyVGG(
  (conv_blvok_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_blvok_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [ ]:
img_batch, label_batch = next(iter(train_dataloader))

img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

model_1.eval()
with torch.inference_mode():
    pred = model_1(img_single.to(device))

print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0208, -0.0020,  0.0095]], device='cuda:0')

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]], device='cuda:0')

Output prediction label:
tensor([0], device='cuda:0')

Actual label:
0


# Creating `engine.py`
This script will contain the train_step(), test_step() and train() functions that we have previously created in other notebook.

In [ ]:
%%writefile going_modular/engine.py

from typing import Dict, List, Tuple
import torch
from tqdm.auto import tqdm

def train_step(model : torch.nn.Module,
               dataloader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               optimizer : torch.optim.Optimizer,
               device : torch.device) -> Tuple[float, float]:

              model.train()

              train_loss, train_acc = 0, 0

              for batch, (X, y) in enumerate(dataloader):
                X, y = X.to(device) , y.to(device)

                y_pred = model(X)

                loss = loss_fn(y_pred, y)
                train_loss += loss.item()

                optimizer.zero_grad()

                loss.backward()

                optimizer.step()

                y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
                train_acc += (y_pred_class == y).sum().item() / len(y_pred)

              train_loss = train_loss / len(dataloader)
              train_acc = train_acc / len(dataloader)
              return train_loss, train_acc

def test_step(model : torch.nn.Module,
              dataloader : torch.utils.data.DataLoader,
              loss_fn : torch.nn.Module,
              device : torch.device) -> Tuple[float, float]:

              model.eval()

              test_loss, test_acc = 0, 0

              with torch.inference_mode():
                for batch, (X, y) in enumerate(dataloader):
                  X, y = X.to(device), y.to(device)

                  test_pred_logits = model(X)

                  loss = loss_fn(test_pred_logits, y)
                  test_loss += loss.item()

                  test_pred_labels = test_pred_logits.argmax(dim=1)
                  test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

              test_loss = test_loss / len(dataloader)
              test_acc = test_acc / len(dataloader)
              return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:

          results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
                }

          for epoch in tqdm(range(epochs)):
              train_loss, train_acc = train_step(model=model,
                                                dataloader=train_dataloader,
                                                loss_fn=loss_fn,
                                                optimizer=optimizer,
                                                device=device)
              test_loss, test_acc = test_step(model=model,
                                              dataloader=test_dataloader,
                                              loss_fn=loss_fn,
                                              device=device)

              print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.4f} | "
                f"train_acc: {train_acc:.4f} | "
                f"test_loss: {test_loss:.4f} | "
                f"test_acc: {test_acc:.4f}"
              )

              results["train_loss"].append(train_loss)
              results["train_acc"].append(train_acc)
              results["test_loss"].append(test_loss)
              results["test_acc"].append(test_acc)

          return results

Overwriting going_modular/engine.py


# Creating `save_model()`

In [ ]:
%%writefile going_modular/utils.py

from pathlib import Path
import torch

def save_model(model : torch.nn.Module,
               target_dir : str,
               model_name : str):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents = True, exist_ok = True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Overwriting going_modular/utils.py


# Creating `train.py`

In [ ]:
%%writefile going_modular/train.py

import os
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils

NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

device = "cuda" if torch.cuda.is_available() else "cpu"

data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/train.py


In [ ]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0968 | train_acc: 0.3672 | test_loss: 1.0934 | test_acc: 0.2604
 20% 1/5 [00:01<00:04,  1.15s/it]Epoch: 2 | train_loss: 1.1134 | train_acc: 0.3047 | test_loss: 1.1396 | test_acc: 0.2604
 40% 2/5 [00:02<00:02,  1.01it/s]Epoch: 3 | train_loss: 1.0895 | train_acc: 0.3125 | test_loss: 1.1227 | test_acc: 0.3125
 60% 3/5 [00:02<00:01,  1.06it/s]Epoch: 4 | train_loss: 1.0798 | train_acc: 0.4336 | test_loss: 1.1276 | test_acc: 0.2083
 80% 4/5 [00:03<00:00,  1.08it/s]Epoch: 5 | train_loss: 1.0453 | train_acc: 0.4141 | test_loss: 1.1238 | test_acc: 0.2083
100% 5/5 [00:05<00:00,  1.05s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
